In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('data/us_election_2020_1st_presidential_debate.csv')
df

,speaker,minute,text
0,Chris Wallace,01:20,Good evening from the Health Education Campus ...
1,Chris Wallace,02:10,This debate is being conducted under health an...
2,Vice President Joe Biden,02:49,"How you doing, man?"
3,President Donald J. Trump,02:51,How are you doing?
4,Vice President Joe Biden,02:51,I’m well.
...,...,...,...
784,Chris Wallace,01:10:43,"Gentlemen, just say that’s the end of it [cros..."
785,President Donald J. Trump,01:10:47,I want to see an honest ballot count.
786,Chris Wallace,01:10:48,We’re going to leave it there-
787,President Donald J. Trump,01:10:49,And I think he does too-


In [7]:
df['speaker'] = df['speaker'].replace({'President Donald J. Trump': 'Trump', 'Vice President Joe Biden': 'Biden'})
df = df.drop('minute', 1)

In [8]:
df

,speaker,text
0,Chris Wallace,Good evening from the Health Education Campus ...
1,Chris Wallace,This debate is being conducted under health an...
2,Biden,"How you doing, man?"
3,Trump,How are you doing?
4,Biden,I’m well.
...,...,...
784,Chris Wallace,"Gentlemen, just say that’s the end of it [cros..."
785,Trump,I want to see an honest ballot count.
786,Chris Wallace,We’re going to leave it there-
787,Trump,And I think he does too-


In [18]:
df['text'] = df['text'].str.lower()
df[df['text'].str.contains('\[crosstalk.*\]', regex=True)]

,speaker,text
58,Chris Wallace,[crosstalk 00:11:26] when i finish i’m going t...
114,Trump,you just lost the left. you agreed with bernie...
133,Biden,he sends out wishful thinking. he has executiv...
221,Trump,"tell that to nancy pelosi, and schumer [crosst..."
269,Biden,"by the way, did you see one of the last big ra..."
...,...,...
760,Trump,"you know it can’t be done. you know it can’t, ..."
761,Biden,mail service delivers [crosstalk 01:07:21] 185...
762,Chris Wallace,"we can keep talking. in eight states, [crossta..."
781,Chris Wallace,i asked you. you had an opportunity to respond...


In [36]:
df['text'] = df['text'].str.replace('\[crosstalk.*\]', '', regex=True)
df['text'] = df['text'].str.strip()

In [32]:
df = df[df['speaker'] != 'Chris Wallace']
df = df.reset_index(drop=True)
df

,speaker,text
0,Biden,"how you doing, man?"
1,Trump,how are you doing?
2,Biden,i’m well.
3,Trump,"thank you very much, chris. i will tell you ve..."
4,Trump,and we won the election and therefore we have ...
...,...,...
559,Trump,it’s already been established. take a look at ...
560,Biden,he has no idea what he’s talking about. here’s...
561,Trump,i want to see an honest ballot cut-
562,Trump,i want to see an honest ballot count.


In [33]:
trump_df = df[df['speaker'] == 'Trump']
biden_df = df[df['speaker'] == 'Biden']

In [34]:
trump_df

,speaker,text
1,Trump,how are you doing?
3,Trump,"thank you very much, chris. i will tell you ve..."
4,Trump,and we won the election and therefore we have ...
6,Trump,"thank you, joe."
10,Trump,there aren’t a hundred million people with pre...
...,...,...
557,Trump,you think that’s good?
559,Trump,it’s already been established. take a look at ...
561,Trump,i want to see an honest ballot cut-
562,Trump,i want to see an honest ballot count.


In [35]:
biden_df

,speaker,text
0,Biden,"how you doing, man?"
2,Biden,i’m well.
5,Biden,"well, first of all, thank you for doing this a..."
7,Biden,the american people have a right to have a say...
8,Biden,"now, what’s at stake here is the president’s m..."
...,...,...
543,Biden,five states have had mail-in ballots for the l...
546,Biden,i am concerned that any court would settle thi...
549,Biden,mail service delivers -
558,Biden,yes. and here’s the deal. we count the ballots...
